Training the model with different input values 

In [ ]:
# importing libraries
import pandas as pd
import numpy as np
from pipeline_config import cfg
from pathlib import Path
import warnings

from xgboost import XGBClassifier, callback
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report, make_scorer, recall_score, precision_score, accuracy_score, log_loss, f1_score
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings(
    "ignore",
    message=".*FrozenEstimator.*sample_weight.*",
    category=UserWarning,
    module="sklearn.calibration"
)
from sklearn.calibration import CalibratedClassifierCV
from sklearn.frozen import FrozenEstimator
from imblearn.over_sampling import SMOTE


In [45]:
# load the processed data

quarter = '2024Q4'

processed_dir = cfg["data"]["processed_dir"]
processed_data_path = processed_dir / cfg["templates"]["processed"].format(quarter=quarter)

print("Loading the processed data !")
df = pd.read_csv(processed_data_path, low_memory=False)

Loading the processed data !


In [46]:
df.head()

,original_interest_rate,current_interest_rate,original_upb,original_loan_term,loan_age,original_loan_to_value_ratio_ltv,number_of_borrowers,debt_to_income_dti,borrower_credit_score_at_origination,number_of_units,...,interest_only_loan_indicator_N,special_eligibility_program_7,special_eligibility_program_F,special_eligibility_program_H,special_eligibility_program_R,property_valuation_method_A,property_valuation_method_C,property_valuation_method_P,property_valuation_method_W,property_state
0,1.226635,1.228290,1.320997,360,-0.656307,95,1,0.641857,-1.534935,1,...,1,1,0,0,0,1,0,0,0,NC
1,1.226635,1.228290,1.320997,360,0.285650,95,1,0.641857,-1.534935,1,...,1,1,0,0,0,1,0,0,0,NC
2,1.226635,1.228290,1.320997,360,1.227606,95,1,0.641857,-1.534935,1,...,1,1,0,0,0,1,0,0,0,NC
3,-1.080236,-1.080436,0.658218,240,-0.656307,61,1,-1.205190,0.567548,1,...,1,1,0,0,0,1,0,0,0,Other
4,-1.080236,-1.080436,0.658218,240,0.285650,61,1,-1.205190,0.567548,1,...,1,1,0,0,0,1,0,0,0,Other


In [47]:
# getting copy of df for train test split
data = df.copy()

data.columns = data.columns.str.replace(r'[^\w]', '_', regex=True)

y = data["default_flag"]
X = data.drop(["default_flag", "property_state"], axis=1)
sf = data["property_state"]


print("Shape of data : ", data.shape)
print("Shape of y : ", y.shape)
print("Shape of X : ", X.shape)
print("Shape of sf : ", sf.shape)

Shape of data :  (508281, 90)
Shape of y :  (508281,)
Shape of X :  (508281, 88)
Shape of sf :  (508281,)


In [48]:
# split the train test data for model

seed = cfg["model"]["random_seed"]

# 60/20/20 stratified
X_tv, X_test, y_tv, y_test, sf_tv, sf_test = train_test_split(
        X, y, sf, test_size=0.2, stratify=y, random_state=seed
    )
X_train, X_val, y_train, y_val, sf_train, sf_val = train_test_split(
    X_tv, y_tv, sf_tv, test_size=0.25, stratify=y_tv, random_state=seed
)

In [49]:
# helper function to evaluation
def evaluate(name, model, X, y, threshold: float=None):

    # Get probabilities
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X)[:, 1]
    else:
        # assume ExponentiatedGradient with model.predictors_ & .weights_
        proba = np.zeros(len(X))
        for mdl, w in zip(model.predictors_, model.weights_):
            proba += w * mdl.predict_proba(X)[:, 1]

    if threshold is None:
        preds = model.predict(X)
    else:
        preds = (proba >= threshold).astype(int)

    print(f"\n--- {name} ---")
    print(f"ROC AUC:           {roc_auc_score(y, proba):.4f}")
    print(f"Avg Precision:    {average_precision_score(y, proba):.4f}")
    print(classification_report(y, preds, digits=4))

In [50]:
# let's have object function for other models to review the performance in other algorithm before tunning the model

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_validate

models = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
}

precision_scorer = make_scorer(precision_score, zero_division=0)
scorers = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': 'roc_auc'
}
results =[]

for name, model in models.items():
    print(f"Training {name}...")
    scores = cross_validate(model, X, y, scoring=scorers)
    
    avg_scores = {
        "Model": name,
        "Accuracy": np.mean(scores['test_accuracy']),
        "Precision": np.mean(scores['test_precision']),
        "Recall": np.mean(scores['test_recall']),
        "F1": np.mean(scores['test_f1']),
        "ROC_AUC": np.mean(scores['test_roc_auc']),
    }
    results.append(avg_scores)

results_df = pd.DataFrame(results).sort_values(by='Recall', ascending=False)
results_df


Training RandomForest...
Training XGBoost...
Training LightGBM...
[LightGBM] [Info] Number of positive: 1799, number of negative: 404825
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012276 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2277
[LightGBM] [Info] Number of data points in the train set: 406624, number of used features: 82
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.004424 -> initscore=-5.416224
[LightGBM] [Info] Start training from score -5.416224
[LightGBM] [Info] Number of positive: 1800, number of negative: 404825
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010819 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2268
[LightGBM] [Info] Number of data points in t

,Model,Accuracy,Precision,Recall,F1,ROC_AUC
2,LightGBM,0.994682,0.207866,0.076908,0.111470,0.902834
1,XGBoost,0.995412,0.330923,0.039566,0.070215,0.897876
0,RandomForest,0.995595,0.822857,0.007561,0.014895,0.811338


In [51]:
# train baseline model 
xgb_cfg = cfg["model"]["xgboost"]

xgb_model = XGBClassifier(
    eval_metric="logloss",
        random_state=cfg["model"]["random_seed"],
        n_estimators=xgb_cfg["n_estimators"],
        max_depth=xgb_cfg["max_depth"],
        learning_rate=xgb_cfg["learning_rate"],
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum()
)

xgb_model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [52]:
# train baseline model
lgb_cfg = cfg["model"]["lightgbm"]

from lightgbm import LGBMClassifier

lgb_model = LGBMClassifier(
    objective="binary",
    random_state=cfg["model"]["random_seed"],
    n_estimators=lgb_cfg["n_estimators"],
    max_depth=lgb_cfg["max_depth"],
    learning_rate=lgb_cfg["learning_rate"],
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum()
)

lgb_model.fit(X_train, y_train)


[LightGBM] [Info] Number of positive: 1349, number of negative: 303619
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010606 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2269
[LightGBM] [Info] Number of data points in the train set: 304968, number of used features: 82
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.004423 -> initscore=-5.416410
[LightGBM] [Info] Start training from score -5.416410
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

LGBMClassifier(colsample_bytree=0.8, max_depth=6, objective='binary',
               random_state=42, scale_pos_weight=np.float64(225.06968124536695),
               subsample=0.8)

In [53]:
# evalaution metrics for the baseline xgboost model
print("▶️  Getting validation score for baseline XGBoostmodel...")
evaluate("Validation score for baseliine model : ", xgb_model, X_val, y_val)
print("-"*80)
print("▶️  Getting validation score for baseline LightGBMmodel...")
evaluate("Validation score for baseliine model : ", lgb_model, X_val, y_val)

▶️  Getting validation score for baseline XGBoostmodel...

--- Validation score for baseliine model :  ---
ROC AUC:           0.9214
Avg Precision:    0.0938
              precision    recall  f1-score   support

           0     0.9988    0.8722    0.9312    101206
           1     0.0259    0.7644    0.0501       450

    accuracy                         0.8717    101656
   macro avg     0.5124    0.8183    0.4907    101656
weighted avg     0.9945    0.8717    0.9273    101656

--------------------------------------------------------------------------------
▶️  Getting validation score for baseline LightGBMmodel...

--- Validation score for baseliine model :  ---
ROC AUC:           0.7596
Avg Precision:    0.0125
              precision    recall  f1-score   support

           0     0.9985    0.7663    0.8671    101206
           1     0.0139    0.7422    0.0273       450

    accuracy                         0.7662    101656
   macro avg     0.5062    0.7543    0.4472    101656
wei

#### Model selection to move forward - XGBoost 

✅ Goal - Primary objective is to identify as many **loan defaults** as possible i.e., maximize recall on the positive class (defaults), while also maintaining reasonable AUC and F1 score. <br><br>

| Metric                     | **XGBoost** | LightGBM |
| -------------------------- | ----------- | -------- |
| **Recall (Class 1)**       | **0.7644**  | 0.7422   |
| **Precision (Class 1)**    | 0.0259      | 0.0139   |
| **F1 Score (Class 1)**     | **0.0501**  | 0.0273   |
| **ROC AUC**                | **0.9214**  | 0.7596   |
| **Avg Precision (PR AUC)** | **0.0938**  | 0.0125   |
| **Accuracy**               | **0.8717**  | 0.7662   |

🧠 Key Observations <br>
- XGBoost achieves higher recall, which aligns with our business need to catch more defaults. <br>

- It also outperforms LightGBM on ROC AUC and average precision (PR AUC), indicating better ranking and probability estimation.
 
- LightGBM had promising CV results, but underperformed on full validation.

- Precision remains low due to class imbalance — this will be addressed in threshold optimization and post-processing stages.

✅ Given it's superior performance on multiple key metrics - especially recall, AUC and F1 Score, moving forward with XGBoost as the core classifier for this ML pipeline.  

In [65]:
# Hyperparameter tuning the XGBoost Model
# object function for hyperparameter tuning and calibration
def tune_params (X_train, y_train):
    seed = 42
    tune_cfg = cfg["model"]["tuning"]
    cv_cfg = cfg["model"]["tuning"]["cv_folds"]
    if cfg["model"]["optimize_for"] == "recall":
        scoring = make_scorer(recall_score)
    else:
        scoring = "average_precision"

    base_model = XGBClassifier(
        eval_metric = "logloss",
        random_state = seed
    )

    # building grid around the config values for tunning
    param_grid = tune_cfg["grid"]

    search = GridSearchCV(
        estimator=base_model,
        param_grid=param_grid,
        cv = cv_cfg,
        scoring=scoring,
        n_jobs=1,
        verbose=1
    )

    search.fit(X_train, y_train)

    print(" Best parameters : ", search.best_params_)

    return search.best_estimator_

def calibrate (model, X_val, y_val):
    # method = "isotonic"
    frozen = FrozenEstimator(model)
    calib = CalibratedClassifierCV(
        estimator=model,
        method="isotonic",
        cv=3
    )
    calib.fit(X_val, y_val)
    return calib

In [67]:
# check evaluation metrics
tuned_model = tune_params(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
 Best parameters :  {'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 400}


In [60]:
cal_model = calibrate(tuned_model, X_val, y_val)
evaluate("Validation score for the calibrated model : ", cal_model, X_val, y_val)


--- Validation score for the calibrated model :  ---
ROC AUC:           1.0000
Avg Precision:    1.0000
              precision    recall  f1-score   support

           0     0.9970    1.0000    0.9985    101206
           1     1.0000    0.3311    0.4975       450

    accuracy                         0.9970    101656
   macro avg     0.9985    0.6656    0.7480    101656
weighted avg     0.9970    0.9970    0.9963    101656



In [61]:
# The score is too low, using SMOTE to reshuffle the data and train again
def resample_train_data(X_train, y_train, sf_train):

    dfs = []
    for group in np.unique(sf_train):
        # select subgroup
        mask = (sf_train == group)
        X_g = X_train[mask]
        y_g = y_train[mask]
        
        # only oversample if positives exist
        if y_g.sum() > 0 and y_g.sum() < len(y_g):
            sm = SMOTE(random_state=42)
            X_res_g, y_res_g = sm.fit_resample(X_g, y_g)
        else:
            # no SMOTE needed if group has only one class
            X_res_g, y_res_g = X_g, y_g
        
        # build a DataFrame to carry group labels
        df_g = pd.DataFrame(X_res_g)
        df_g["_target"] = y_res_g
        df_g["_sf"] = group
        dfs.append(df_g)

    # combine all groups
    df_res = pd.concat(dfs, ignore_index=True)

    # split back out
    X_res = df_res.drop(columns=["_target", "_sf"]).values
    y_res = df_res["_target"].values
    sf_res = df_res["_sf"].values

    return X_res, y_res, sf_res

In [62]:
# extracting train data using reshuffling 
X_res, y_res, sf_res = resample_train_data(X_train, y_train, sf_train)

In [63]:
# retraining again with reshuffled dataset 
tuned_model_res = tune_params(X_res, y_res)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
 Best parameters :  {'learning_rate': 0.2, 'max_depth': 4, 'n_estimators': 400}


In [66]:

cal_model_res = calibrate(tuned_model_res, X_val, y_val)
evaluate("Validation score for the calibrated model : ", cal_model_res, X_val, y_val)


--- Validation score for the calibrated model :  ---
ROC AUC:           0.9999
Avg Precision:    0.9890
              precision    recall  f1-score   support

           0     0.9967    1.0000    0.9983    101206
           1     1.0000    0.2444    0.3929       450

    accuracy                         0.9967    101656
   macro avg     0.9983    0.6222    0.6956    101656
weighted avg     0.9967    0.9967    0.9956    101656



In [70]:
# check for predictions 
prediction_cal = cal_model_res.predict(X_test)

In [72]:
print("Prediction on test set !")
print(classification_report(y_test, prediction_cal))
print("ROC AUC:", roc_auc_score(y_test, cal_model_res.predict_proba(X_test)[:, 1]))

Prediction on test set !
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    101207
           1       0.71      0.01      0.02       450

    accuracy                           1.00    101657
   macro avg       0.85      0.51      0.51    101657
weighted avg       0.99      1.00      0.99    101657

ROC AUC: 0.8985506821552747


We can observe the sharp dive of  recall for type 1. Out of 450 only ~5 defaults were caught. This signals the classic accuracy trap due to class imbalance. <br>
<br>
So despite having, <br> 
&nbsp;&nbsp; - Perfect accuracy (100%) <br>
&nbsp;&nbsp; - Perfect precision for class 0 <br>
&nbsp;&nbsp; - AUC of 0.89 (not bad) <br>

It only captured 1% of defaulters — which means: <br>
&nbsp;&nbsp; - 🔥 Model is essentially predicting "no default" for everyone. <br>

And yet ROC AUC looks good — why? <br>
&nbsp;&nbsp; - Because ROC AUC is threshold-independent — it only reflects ranking, not classification. <br>

What next ?? <br>
&nbsp; - Introduce precision-recall trade off to pick the threshold which<br>    
&nbsp;&nbsp;&nbsp;&nbsp; -> maximizes f1/recall <br>
&nbsp;&nbsp;&nbsp;&nbsp; -> or achieves minimum required recall (eg. > 2.0)

Implementing quantile based cutoff 

In [ ]:
# computing full precision-recall tradeoff on the validation set using calibrated model
proba_val = cal_model.predict_proba(X_val)[:, 1]

# implementing quantile based cutoff
# computing cutoff for top k% as defined in config
pct_grid      = [0.01, 0.02, 0.05, 0.10, 0.15]
precision_min = 0.4
pct_max       = 0.05

results = []
for pct in pct_grid:
    # skip any pct beyond the max investigation budget
    if pct > pct_max:
        continue

    # compute the cutoff for this top‐pct
    thr   = np.percentile(proba_val, 100 * (1 - pct))
    preds = (proba_val >= thr).astype(int)
    prec  = precision_score(y_val, preds, zero_division=0)
    rec   = recall_score(y_val, preds)
    results.append({
        "top_pct":  pct,
        "threshold": thr,
        "precision": prec,
        "recall":    rec
    })

df_grid = pd.DataFrame(results)

# filter by pct_max only
filtered = df_grid[df_grid.top_pct <= pct_max]
print(f"\nCandidates with top_pct ≤ {pct_max}:")
print(filtered.to_string(index=False))

candidates = filtered[filtered.precision >= precision_min]
if candidates.empty:
    print(f"\n⚠️  No slice meets precision ≥ {precision_min}; falling back to pct_max only")
    best = filtered.loc[filtered.recall.idxmax()]
else:
    best = candidates.loc[candidates.recall.idxmax()]

sel_pct = best.top_pct
sel_threshold = best.threshold

print(f"\n ▶▶▶ Chosen top_pct: {sel_pct:.3f} (≤ {pct_max}), threshold: {sel_threshold:.6f}")
print(f" ▶▶▶ Precision: {best.precision:.4f} (≥ {precision_min}), Recall: {best.recall:.4f}")
print("="*60)


Candidates with top_pct ≤ 0.05:
 top_pct  threshold  precision  recall
    0.01   0.040127   0.432277     1.0
    0.02   0.027975   0.220157     1.0
    0.05   0.016447   0.088530     1.0

 ▶▶▶ Chosen top_pct: 0.010 (≤ 0.05), threshold: 0.040127
 ▶▶▶ Precision: 0.4323 (≥ 0.4), Recall: 1.0000


: 

📌 Observation: <br>
🎯 What We Achieved <br>
By flagging the top 1% of loan applicants with the highest model-predicted risk (i.e., probability > 4.01%), <br>
we achieved:
| Metric            | Value | Business Meaning                                                                 |
| ----------------- | ----- | -------------------------------------------------------------------------------- |
| **Recall**        | 100%  | ✅ **All** defaulters were caught by the model — no missed risk cases             |
| **Precision**     | 43.2% | ⚠️ Among the flagged loans, \~43% are true defaulters; \~57% are false positives |
| **Top % Flagged** | 1%    | Out of every 100 loans, only **1** is flagged for review/intervention            |


<br>

📦 What This Means Operationally <br>

Assuming a validation/test set of 101,657 loans, of which 450 are defaulters:

- The model correctly identified all 450 defaulters.

- It flagged 1% of loans, i.e., ~1,016 applicants.

- Of these, 450 are true defaulters → we intervened on just 1,016 loans to catch all 450 risky ones.

📉 Without this model or with a default threshold of 0.5, only ~1–2% of defaulters were being caught.

<br>
💰 Cost–Benefit Framing for Stakeholders <br>
"To catch 100% of defaults, we only need to manually review 1% of all applicants." <br>

| Factor                   | Before (default 0.5) | After (optimized 0.0401)    |
| ------------------------ | -------------------- | --------------------------- |
| Defaulters Caught        | \~1–2%               | **100%** ✅                  |
| Applicants Flagged       | Negligible (\~10)    | \~1,016                     |
| False Positives Reviewed | \~566                | Acceptable operational load |
| Missed Defaulters        | \~440+               | **0**                       |

This trade-off is justifiable and practical:

- We sacrifice some precision (we'll review some false positives)

- But we eliminate missed defaulters, which is crucial for:

&nbsp;&nbsp;&nbsp; - Reducing defaulted capital exposure

&nbsp;&nbsp;&nbsp; - Meeting regulatory stress test expectations

&nbsp;&nbsp;&nbsp;  - Protecting the loan book health

We can wrap the fairlearn over this selected model